In [1]:
import sys
import logging
import pickle

import numpy as np
import matplotlib.pyplot as plt
from joblib import hash, dump, load
import os

from deer.default_parser import process_args
from deer.agent import NeuralAgent
from deer.learning_algos.CRAR_torch import CRAR
from figure8_env import MyEnv as figure8_env
import deer.experiment.base_controllers as bc

from deer.policies import EpsilonGreedyPolicy, FixedFigure8Policy

In [2]:
figure8_give_rewards = True
nn_yaml = 'network_noconv.yaml'
high_dim_obs = False
internal_dim = 10
fname = '2d_obs'
set_network = None #['expanded_tcm', 15, True]

In [3]:
class Defaults:
    # ----------------------
    # Setup Parameters (copied for convenience)
    # ----------------------
    figure8_give_rewards = figure8_give_rewards
    nn_yaml = nn_yaml
    high_dim_obs = high_dim_obs
    internal_dim = internal_dim
    fname = fname
    
    # ----------------------
    # Experiment Parameters
    # ----------------------
    steps_per_epoch = 2500
    epochs = 50
    steps_per_test = 1000
    period_btw_summary_perfs = 1

    # ----------------------
    # Temporal Processing Parameters
    # ----------------------
    nstep = 15
    nstep_decay = 0.8
    expand_tcm = True
    encoder_type = 'regular'
    
    # ----------------------
    # Environment Parameters
    # ----------------------
    frame_skip = 2
    show_rewards = False

    # ----------------------
    # DQN Agent parameters:
    # ----------------------
    learning_rate = 1*1E-4
    learning_rate_decay = 1.0
    discount = 0.9
    epsilon_start = 1.0
    epsilon_min = 1.0
    epsilon_decay = 1000
    update_frequency = 1
    replay_memory_size = 100000 #50000
    batch_size = 64
    freeze_interval = 1000
    deterministic = False
    
    # ----------------------
    # Learning algo parameters
    # ----------------------
    # T, entropy_neighbor, entropy_random, volume, gamma, R, Q, variational
    #loss_weights = [5E-3, 1E-3, 5E-3, 5E-3, 5E-3, 5E-3, 1.]
    loss_weights = [0, 0, 0, 0, 0, 0, 1., 0.]
    #loss_weights = [0., 0., 0., 0., 0., 0., 1., 2E-4]
    #loss_weights = [5E-3, 1E-3, 5E-3, 5E-3, 5E-3, 5E-3, 0., 0.]

In [4]:
parameters = Defaults()
with open(f'params/{fname}.p', 'wb') as f:
    pickle.dump(parameters, f)

In [5]:
if parameters.deterministic:
    rng = np.random.RandomState(123456)
else:
    rng = np.random.RandomState()

# --- Instantiate environment ---
env = figure8_env(
    give_rewards=figure8_give_rewards,
    intern_dim=internal_dim,
    high_dim_obs=high_dim_obs,
    show_rewards=parameters.show_rewards
    )

# --- Instantiate learning_algo ---
learning_algo = CRAR(
    env,
    parameters.freeze_interval,
    parameters.batch_size,
    rng,
    high_int_dim=False,
    internal_dim=internal_dim, lr=parameters.learning_rate,
    nn_yaml=nn_yaml, double_Q=True,
    loss_weights=parameters.loss_weights,
    nstep=parameters.nstep, nstep_decay=parameters.nstep_decay,
    encoder_type=parameters.encoder_type,
    expand_tcm=parameters.expand_tcm
    )

if figure8_give_rewards:
    train_policy = EpsilonGreedyPolicy(
        learning_algo, env.nActions(), rng, 0.2,
        consider_valid_transitions=False
        )
    test_policy = EpsilonGreedyPolicy(
        learning_algo, env.nActions(), rng, 0.
        )
else:
    train_policy = FixedFigure8Policy.FixedFigure8Policy(
        learning_algo, env.nActions(), rng, epsilon=0.2,
        height=env.HEIGHT, width=env.WIDTH
        )
    test_policy = FixedFigure8Policy.FixedFigure8Policy(
        learning_algo, env.nActions(), rng,
        height=env.HEIGHT, width=env.WIDTH
        )

# --- Instantiate agent ---
agent = NeuralAgent(
    env, learning_algo,
    parameters.replay_memory_size,
    1, parameters.batch_size, rng,
    train_policy=train_policy, test_policy=test_policy)
if set_network is not None:
    agent.setNetwork(
        f'{set_network[0]}/fname', nEpoch=set_network[1],
        encoder_only=set_network[2]
        )

agent.run(10, 500)
print("end gathering data")

# --- Bind controllers to the agent ---
# Before every training epoch (periodicity=1), we want to print a summary of the agent's epsilon, discount and 
# learning rate as well as the training epoch number.
agent.attach(bc.VerboseController(
    evaluate_on='epoch', 
    periodicity=1))

# Learning rate may follow a scheduler
agent.attach(bc.LearningRateController(
    initial_learning_rate=parameters.learning_rate, 
    learning_rate_decay=parameters.learning_rate_decay,
    periodicity=1))

# During training epochs, we want to train the agent after every [parameters.update_frequency] action it takes.
# Plus, we also want to display after each training episode (!= than after every training) the average bellman
# residual and the average of the V values obtained during the last episode, hence the two last arguments.
agent.attach(bc.TrainerController(
    evaluate_on='action', 
    periodicity=parameters.update_frequency, 
    show_episode_avg_V_value=True, 
    show_avg_Bellman_residual=True))

# We wish to discover, among all versions of our neural network (i.e., after every training epoch), which one 
# has the highest validation score.
# To achieve this goal, one can use the FindBestController along with an InterleavedTestEpochControllers. It is 
# important that the validationID is the same than the id argument of the InterleavedTestEpochController.
# The FindBestController will dump on disk the validation scores for each and every network, as well as the 
# structure of the neural network having the best validation score. These dumps can then used to plot the evolution 
# of the validation and test scores (see below) or simply recover the resulting neural network for your 
# application.
agent.attach(bc.FindBestController(
    validationID=figure8_env.VALIDATION_MODE,
    testID=None,
    unique_fname=fname, savefrequency=5))

# All previous controllers control the agent during the epochs it goes through. However, we want to interleave a 
# "validation epoch" between each training epoch. For each validation epoch, we want also to display the sum of all 
# rewards obtained, hence the showScore=True. Finally, we want to call the summarizePerformance method of ALE_env 
# every [parameters.period_btw_summary_perfs] *validation* epochs.
agent.attach(bc.InterleavedTestEpochController(
    id=figure8_env.VALIDATION_MODE, 
    epoch_length=parameters.steps_per_test,
    periodicity=1,
    show_score=True,
    summarize_every=1,
    unique_fname=fname))

end gathering data


In [6]:
try:
    os.mkdir("params")
except Exception:
    pass
dump(vars(parameters), "params/" + fname + ".jldump")
#agent.gathering_data=False
if set_network is not None:
    agent.setNetwork(
        f'{set_network[0]}/fname', nEpoch=set_network[1],
        encoder_only=set_network[2]
        )
agent.run(parameters.epochs, parameters.steps_per_epoch)

# --- Show results ---
basename = "scores/" + fname
scores = load(basename + "_scores.jldump")
print(scores)

Printing a few elements useful for debugging:
actions_val[0], rewards_val[0], terminals_val[0]
0 0.0 0.0
Es[0], TEs[0], Esp_[0]
tensor([ 0.0359, -0.2225,  0.3165,  0.1578,  0.2568,  0.0076, -0.0083,  0.3265,
        -0.0484,  0.0728]) tensor([ 0.1811, -0.0303,  0.0336,  0.3680, -0.1580, -0.2024,  0.1085,  0.1297,
        -0.1646, -0.2863]) tensor([ 0.0994, -0.2553,  0.2563,  0.0890,  0.3583, -0.0465, -0.0044,  0.3508,
        -0.1148,  0.0970])
R[0]
tensor([0.1647], grad_fn=<SelectBackward0>)
LOSSES
T = 0.058171156398952005; R = 0.015935857269912958;                 Gamma = 0.689298388838768; Q = 0.002145316639856901;
Entropy Neighbor = 0.7433496443033218;                 Entropy Random = 0.6659789493083954;                 Volume = 0.0; VAE = 0.0
Printing a few elements useful for debugging:
actions_val[0], rewards_val[0], terminals_val[0]
3 0.0 0.0
Es[0], TEs[0], Esp_[0]
tensor([ 0.1715, -0.1128,  0.3757,  0.1519, -0.0440, -0.1954, -0.1946,  0.6030,
         0.4154,  0.0104]) tensor(

/Users/chingfang/miniforge3/envs/auxrl/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Testing score per episode (id: 0) is 0.0 (average over 1 episode(s))
== Mean score per episode is 0.0 over 1 episodes ==


/Users/chingfang/Code/deer/examples/test_CRAR/figure8_env.py:320: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/Users/chingfang/Code/deer/examples/test_CRAR/figure8_env.py:366: RuntimeWarning: invalid value encountered in divide
  dist_matrix = dist_matrix/np.nanpercentile(dist_matrix.flatten(), 99)
/Users/chingfang/Code/deer/examples/test_CRAR/figure8_env.py:403: RuntimeWarning: All-NaN axis encountered
  ylim_max = np.nanmax(self._separability_tracking)*1.1


Printing a few elements useful for debugging:
actions_val[0], rewards_val[0], terminals_val[0]
3 0.0 0.0
Es[0], TEs[0], Esp_[0]
tensor([-0.1182,  0.0510,  0.0382,  0.0831, -0.0323, -0.1822, -0.1320,  0.1309,
         0.0105,  0.0749]) tensor([ 0.0282,  0.2775, -0.2436,  0.2987, -0.4537, -0.3863, -0.0272, -0.0367,
        -0.0836, -0.2879]) tensor([-0.1343,  0.0389,  0.0633,  0.1093, -0.0266, -0.1729, -0.1053,  0.1553,
         0.0451,  0.0736])
R[0]
tensor([0.1159], grad_fn=<SelectBackward0>)
LOSSES
T = 0.06037188592553139; R = 0.01337101586908102;                 Gamma = 0.7014917446374893; Q = 0.000675403090499458;
Entropy Neighbor = 0.7835826264619827;                 Entropy Random = 0.584825124502182;                 Volume = 0.00019380903616547583; VAE = 0.0
Printing a few elements useful for debugging:
actions_val[0], rewards_val[0], terminals_val[0]
1 0.0 0.0
Es[0], TEs[0], Esp_[0]
tensor([ 0.0269,  0.0794,  0.1409,  0.1096, -0.0972, -0.0938, -0.1799,  0.0978,
         0.0031, 

...

/Users/chingfang/Code/deer/examples/test_CRAR/figure8_env.py:320: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


Printing a few elements useful for debugging:
actions_val[0], rewards_val[0], terminals_val[0]
0 0.0 0.0
Es[0], TEs[0], Esp_[0]
tensor([-0.1052, -0.1587,  0.2796,  0.1180,  0.6291,  0.0534,  0.4401,  0.4956,
        -0.1199,  0.0304]) tensor([ 3.7895e-02,  3.1473e-02,  2.5403e-04,  3.2462e-01,  2.1288e-01,
        -1.5733e-01,  5.6115e-01,  2.8208e-01, -2.2633e-01, -3.2617e-01]) tensor([-0.1818, -0.1876,  0.1957,  0.1214,  0.6138,  0.0454,  0.4306,  0.4427,
        -0.1810, -0.0103])
R[0]
tensor([0.1860], grad_fn=<SelectBackward0>)
LOSSES
T = 0.10077220456302166; R = 0.08638393099233509;                 Gamma = 0.7176505808830261; Q = 0.0040960134541383015;
Entropy Neighbor = 0.29477955934405325;                 Entropy Random = 0.10055364229530096;                 Volume = 0.050850992772728205; VAE = 0.0
Printing a few elements useful for debugging:
actions_val[0], rewards_val[0], terminals_val[0]
3 0.0 0.0
Es[0], TEs[0], Esp_[0]
tensor([ 0.0274,  0.1747,  0.4544,  0.1425,  0.5620,  0

/Users/chingfang/Code/deer/examples/test_CRAR/figure8_env.py:320: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


Printing a few elements useful for debugging:
actions_val[0], rewards_val[0], terminals_val[0]
2 0.0 0.0
Es[0], TEs[0], Esp_[0]
tensor([-0.3435,  0.1553,  0.4260,  0.0034,  0.2907,  0.1564,  0.0185,  0.3769,
        -0.3444,  0.0965]) tensor([-0.1802,  0.4142,  0.1465,  0.2709, -0.1538, -0.0543,  0.0732,  0.2169,
        -0.4444, -0.2485]) tensor([-0.2535, -0.0713,  0.4425,  0.1111,  0.4771,  0.2153,  0.2541,  0.3408,
        -0.3580,  0.1952])
R[0]
tensor([0.1103], grad_fn=<SelectBackward0>)
LOSSES
T = 0.1004647610783577; R = 0.08343044790253043;                 Gamma = 0.7171525371074676; Q = 0.004416846320731566;
Entropy Neighbor = 0.294104270786047;                 Entropy Random = 0.1014782331287861;                 Volume = 0.04861234221607447; VAE = 0.0
Printing a few elements useful for debugging:
actions_val[0], rewards_val[0], terminals_val[0]
0 0.0 0.0
Es[0], TEs[0], Esp_[0]
tensor([0.0589, 0.3270, 0.4028, 0.1434, 0.3096, 0.0070, 0.1629, 0.4045, 0.0948,
        0.0339]) tens


KeyboardInterrupt



In [ ]:
agent._learning_algo.device

## Visualize performance

In [ ]:
agent.setNetwork(fname, nEpoch=7)

In [ ]:
agent._in_episode = True
agent._mode = 0 # Testing mode with plan_depth=0
initState = env.reset(agent._mode)
inputDims = env.inputDimensions()

for i in range(len(inputDims)):
    if inputDims[i][0] > 1:
        agent._state[i][1:] = initState[i][1:]
agent._Vs_on_last_episode = []
is_terminal = False
reward = 0

### Frame by frame

In [ ]:
%matplotlib inline

for i in range(100):
    obs = env.observe()
    _obs = obs[0].reshape((env.WIDTH, env.HEIGHT))
    plt.figure()
    plt.imshow(np.flip(_obs.squeeze()))
    plt.show()
    for i in range(len(obs)):
        agent._state[i][0:-1] = agent._state[i][1:]
        agent._state[i][-1] = obs[i]
    V, action, reward, _ = agent._step()
    print(action)
    agent._Vs_on_last_episode.append(V)
    is_terminal = env.inTerminalState()
    if is_terminal: break

### As animation

In [ ]:
%matplotlib notebook

import numpy as np 
import matplotlib.pyplot as plt
from IPython.display import HTML
import matplotlib.animation as animation

fig, ax = plt.subplots(1,1)
obs = env.observe()
_obs = obs[0].reshape((env.WIDTH, env.HEIGHT))
_obs = np.flip(_obs.squeeze())
ax.set_xticks([])
ax.set_yticks([])
im = ax.imshow(np.zeros(_obs.shape))

def init():
    plt.cla()
    im = ax.imshow(_obs)
    return [im]

def animate(i, *args, **kwargs):
    plt.cla()
    obs = env.observe()
    _obs = obs[0].reshape((env.WIDTH, env.HEIGHT))
    _obs = np.flip(_obs.squeeze())
    im = ax.imshow(_obs)
    for i in range(len(obs)):
        agent._state[i][0:-1] = agent._state[i][1:]
        agent._state[i][-1] = obs[i]
        V, action, reward, _ = agent._step()
        agent._Vs_on_last_episode.append(V)
    return [im]

ani = animation.FuncAnimation(fig, animate, init_func=init, 
     frames=100, blit=False, repeat=True)
ani.save(f'figs/{fname}/behavior.gif', writer="ffmpeg", fps = 15)
fig.show()